In [105]:
from pyspark.sql import SparkSession
# 스파크 환경 설정 객체 생성
spark = SparkSession.builder.appName("241206_03_summary").getOrCreate()

In [106]:
df = spark.read.format('csv').load('data/2015-summary.csv', inferSchema=True, header=True)

In [107]:
df.schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,IntegerType,true)))

In [108]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



In [109]:
df.take(5)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [110]:
df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [111]:
df.select("DEST_COUNTRY_NAME").show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
|    United States|
|            Egypt|
|    United States|
+-----------------+
only showing top 5 rows



In [112]:
df.createOrReplaceTempView("mobility_data")

In [113]:
spark.sql("select * from mobility_data").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [114]:
df.count()

256

In [115]:
df_dup2 = df.select('DEST_COUNTRY_NAME').dropDuplicates().cache()

In [116]:
df_dup2.show()

+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|            Anguilla|
|              Russia|
|            Paraguay|
|             Senegal|
|              Sweden|
|            Kiribati|
|              Guyana|
|         Philippines|
|            Djibouti|
|            Malaysia|
|           Singapore|
|                Fiji|
|              Turkey|
|                Iraq|
|             Germany|
|              Jordan|
|               Palau|
|Turks and Caicos ...|
|              France|
|              Greece|
+--------------------+
only showing top 20 rows



In [117]:
df_dup3 = df.selcet('DEST_COUNTRY_NAME').dropDuplicates().cache()
df_dup3.show()

AttributeError: 'DataFrame' object has no attribute 'selcet'

In [118]:
df_dup.distinct().count()

NameError: name 'df_dup' is not defined

In [119]:
from pyspark.sql.functions import expr

In [120]:
df3 = df.withColumn('withInCountry', expr('ORIGIN_COUNTRY_NAME==DEST_COUNTRY_NAME'))

In [121]:
df3.show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withInCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [122]:
df4 = df.withColumn('category', expr('CASE WHEN count<10 THEN "under" WHEN count>=10 THEN "upper" END'))

In [123]:
df4.show(5)

+-----------------+-------------------+-----+--------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|category|
+-----------------+-------------------+-----+--------+
|    United States|            Romania|   15|   upper|
|    United States|            Croatia|    1|   under|
|    United States|            Ireland|  344|   upper|
|            Egypt|      United States|   15|   upper|
|    United States|              India|   62|   upper|
+-----------------+-------------------+-----+--------+
only showing top 5 rows



In [124]:
df5 = df4.withColumn('withINCountry', expr('ORIGIN_COUNTRY_NAME==DEST_COUNTRY_NAME'))

In [125]:
df5.show(5)

+-----------------+-------------------+-----+--------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|category|withINCountry|
+-----------------+-------------------+-----+--------+-------------+
|    United States|            Romania|   15|   upper|        false|
|    United States|            Croatia|    1|   under|        false|
|    United States|            Ireland|  344|   upper|        false|
|            Egypt|      United States|   15|   upper|        false|
|    United States|              India|   62|   upper|        false|
+-----------------+-------------------+-----+--------+-------------+
only showing top 5 rows



In [126]:
df5.groupBy("withInCountry").count().show()

+-------------+-----+
|withInCountry|count|
+-------------+-----+
|         true|    1|
|        false|  255|
+-------------+-----+



In [127]:
df6 = df5.where('count<5')

In [128]:
df6.show()

+--------------------+-------------------+-----+--------+-------------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|category|withINCountry|
+--------------------+-------------------+-----+--------+-------------+
|       United States|            Croatia|    1|   under|        false|
|       United States|          Singapore|    1|   under|        false|
|             Moldova|      United States|    1|   under|        false|
|               Malta|      United States|    1|   under|        false|
|             Algeria|      United States|    4|   under|        false|
|       United States|          Gibraltar|    1|   under|        false|
|Saint Vincent and...|      United States|    1|   under|        false|
|            Suriname|      United States|    1|   under|        false|
|       United States|             Cyprus|    1|   under|        false|
|       United States|           Malaysia|    3|   under|        false|
|            Thailand|      United States|    3|   under|       

In [129]:
df6.count()

46

In [130]:
df7=df5.where('count<5').where('ORIGIN_COUNTRY_NAME != "United States"')
df7.show()

+-----------------+-------------------+-----+--------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|category|withINCountry|
+-----------------+-------------------+-----+--------+-------------+
|    United States|            Croatia|    1|   under|        false|
|    United States|          Singapore|    1|   under|        false|
|    United States|          Gibraltar|    1|   under|        false|
|    United States|             Cyprus|    1|   under|        false|
|    United States|           Malaysia|    3|   under|        false|
|    United States|            Vietnam|    2|   under|        false|
|    United States|            Estonia|    1|   under|        false|
|    United States|            Hungary|    3|   under|        false|
|    United States|           Thailand|    4|   under|        false|
|    United States|            Liberia|    2|   under|        false|
|    United States|              Malta|    2|   under|        false|
|    United States|          Lithu

In [131]:
spark.sql("select * from mobility_data").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [132]:
#upper 인 ORIGIN_COUNTRY_NAEM별 평균 카운트

In [133]:
#카운트가 200 이상인 ORIGIN_COUNTRY_NAME

In [134]:
#국내 여행이 아니면서 가장 횟수가 많은 ORIGIN_COUNTRY_NAME top 10을 추출해보세요

In [135]:
#국내 여행이 아니면서 가장 횟수가 적은 ORIGIN_COUNTRY_NAME top 10을 추출해보세요

In [136]:
#도착국가별 count 총합이 가장 많은 top10을 추출해 보세요

In [1]:
spark.stop()

NameError: name 'spark' is not defined

In [137]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SecondSparkSessionApp").getOrCreate()

In [138]:
df = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/emp.csv')


In [139]:
df.printSchema()

root
 |-- empno: integer (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: integer (nullable = true)
 |-- hiredate: string (nullable = true)
 |-- sal: integer (nullable = true)
 |-- comm: integer (nullable = true)
 |-- deptno: integer (nullable = true)



In [140]:
df.show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|null|    20|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|null|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|null|    20|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|null|    30|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|null|    20|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|
| 9292|  JACK|

In [141]:
df.select('ename','deptno').show()

+------+------+
| ename|deptno|
+------+------+
| SMITH|    20|
| ALLEN|    30|
|  WARD|    30|
| JONES|    20|
|MARTIN|    30|
| BLAKE|    30|
| CLARK|    10|
| SCOTT|    20|
|  KING|    10|
|TURNER|    30|
| ADAMS|    20|
| JAMES|    30|
|  FORD|    20|
|MILLER|    10|
|  JACK|    70|
+------+------+



In [142]:
df.select('ename', 'deptno').where('deptno=20').show()

+-----+------+
|ename|deptno|
+-----+------+
|SMITH|    20|
|JONES|    20|
|SCOTT|    20|
|ADAMS|    20|
| FORD|    20|
+-----+------+



In [143]:
from pyspark.sql.functions import count

In [144]:
df.select(count('job')).show()

+----------+
|count(job)|
+----------+
|        15|
+----------+



In [145]:
df.selectExpr().show()

++
||
++
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
++



In [146]:
df.select('job').distinct().count()

5

In [147]:
from pyspark.sql.functions import countDistinct

In [148]:
df.select(countDistinct('job')).show()

+-------------------+
|count(DISTINCT job)|
+-------------------+
|                  5|
+-------------------+



In [149]:
from pyspark.sql.functions import countDistinct, approx_count_distinct

In [150]:
df.select(approx_count_distinct('job',0.1)).show() #성능면에서 유리한 연산

+--------------------------+
|approx_count_distinct(job)|
+--------------------------+
|                         5|
+--------------------------+



In [151]:
from pyspark.sql.functions import first, last, min, max, sum, avg, expr

In [152]:
df.select(first('ename'), last('ename')).show()

+------------+-----------+
|first(ename)|last(ename)|
+------------+-----------+
|       SMITH|       JACK|
+------------+-----------+



In [153]:
df.select(min('sal'), max('sal')).show()

+--------+--------+
|min(sal)|max(sal)|
+--------+--------+
|     800|    5000|
+--------+--------+



In [154]:
df.select(count('empno'), count('*'),max('ename'),min('ename')).show()

+------------+--------+----------+----------+
|count(empno)|count(1)|max(ename)|min(ename)|
+------------+--------+----------+----------+
|          15|      15|      WARD|     ADAMS|
+------------+--------+----------+----------+



In [155]:
df.select(sum('sal')).show()

+--------+
|sum(sal)|
+--------+
|   32225|
+--------+



In [156]:
df.selectExpr('sum(distinct sal)').show()

+-----------------+
|sum(DISTINCT sal)|
+-----------------+
|            27975|
+-----------------+



In [157]:
dfs = df.select(count('sal').alias('total_tx'),
                sum('sal').alias('total_salary'),
                #avg('sal').alias('avg_salary')
                expr('mean(sal)').alias('mean_salary')
               )
dfs.show()

+--------+------------+------------------+
|total_tx|total_salary|       mean_salary|
+--------+------------+------------------+
|      15|       32225|2148.3333333333335|
+--------+------------+------------------+



In [158]:
df

DataFrame[empno: int, ename: string, job: string, mgr: int, hiredate: string, sal: int, comm: int, deptno: int]

In [159]:
df.groupBy('job').count().show()

+---------+-----+
|      job|count|
+---------+-----+
|  ANALYST|    2|
| SALESMAN|    4|
|    CLERK|    5|
|  MANAGER|    3|
|PRESIDENT|    1|
+---------+-----+



In [160]:
dfs = df.groupBy('job').agg(expr('avg(sal) as SAL_AVG'))

In [161]:
dfs.show()

+---------+------------------+
|      job|           SAL_AVG|
+---------+------------------+
|  ANALYST|            3000.0|
| SALESMAN|            1400.0|
|    CLERK|            1470.0|
|  MANAGER|2758.3333333333335|
|PRESIDENT|            5000.0|
+---------+------------------+



In [162]:
df.groupBy('job').agg(expr('stddev_pop(sal) as SAL_STDEV')).show()

+---------+------------------+
|      job|         SAL_STDEV|
+---------+------------------+
|  ANALYST|               0.0|
| SALESMAN|154.11035007422439|
|    CLERK| 880.6815542521599|
|  MANAGER|223.91714737574006|
|PRESIDENT|               0.0|
+---------+------------------+



In [163]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank
windowspec = Window.orderBy(desc('sal'))

In [164]:
salAllRank = rank().over(windowspec)

In [165]:
salAllRank

Column<'RANK() OVER (ORDER BY sal DESC NULLS LAST unspecifiedframe$())'>

In [166]:
#데이터프레임에 컬럼으로 추가 > 액션
df.withColumn("salary_rank", salAllRank).show()

+-----+------+---------+----+----------+----+----+------+-----------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|salary_rank|
+-----+------+---------+----+----------+----+----+------+-----------+
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|          1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|null|    70|          2|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|          3|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|null|    20|          3|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|null|    20|          5|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|null|    30|          6|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|          7|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|          8|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|          9|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|         10|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|         11|
| 7654|MARTIN| SALES

24/12/09 14:27:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [167]:
from pyspark.sql.functions import col
df.select( 'empno', salAllRank.alias('salary_rank') ).show()

+-----+-----------+
|empno|salary_rank|
+-----+-----------+
| 7839|          1|
| 9292|          2|
| 7788|          3|
| 7902|          3|
| 7566|          5|
| 7698|          6|
| 7782|          7|
| 7499|          8|
| 7844|          9|
| 7934|         10|
| 7521|         11|
| 7654|         11|
| 7876|         13|
| 7900|         14|
| 7369|         15|
+-----+-----------+



24/12/09 14:27:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [168]:
windowspec = Window.orderBy(desc('sal'))
windowspec =Window.partitionBy('job').orderBy(desc('sal'))

In [169]:
salJobRank = rank().over(windowspec)

In [170]:
windowspec

In [171]:
df.select(
    'job',
    'ename',
    'sal',
    salJobRank.alias('salJobRank')  # rank 결과에 별칭 부여
).show()

+---------+------+----+----------+
|      job| ename| sal|salJobRank|
+---------+------+----+----------+
|  ANALYST| SCOTT|3000|         1|
|  ANALYST|  FORD|3000|         1|
| SALESMAN| ALLEN|1600|         1|
| SALESMAN|TURNER|1500|         2|
| SALESMAN|  WARD|1250|         3|
| SALESMAN|MARTIN|1250|         3|
|    CLERK|  JACK|3200|         1|
|    CLERK|MILLER|1300|         2|
|    CLERK| ADAMS|1100|         3|
|    CLERK| JAMES| 950|         4|
|    CLERK| SMITH| 800|         5|
|  MANAGER| JONES|2975|         1|
|  MANAGER| BLAKE|2850|         2|
|  MANAGER| CLARK|2450|         3|
|PRESIDENT|  KING|5000|         1|
+---------+------+----+----------+



In [172]:
#부서별 급여 순위 생성부서별 급여 순위 생성
dept_window_spec = Window.partitionBy('deptno').orderBy(desc('sal'))

In [173]:
new_df = df.withColumn( 'dept_salary_rank', rank().over(dept_window_spec))
new_df.select(
    'ename', 'deptno', 'sal', 'dept_salary_rank'
).show()

+------+------+----+----------------+
| ename|deptno| sal|dept_salary_rank|
+------+------+----+----------------+
| SCOTT|    20|3000|               1|
|  FORD|    20|3000|               1|
| JONES|    20|2975|               3|
| ADAMS|    20|1100|               4|
| SMITH|    20| 800|               5|
|  KING|    10|5000|               1|
| CLARK|    10|2450|               2|
|MILLER|    10|1300|               3|
|  JACK|    70|3200|               1|
| BLAKE|    30|2850|               1|
| ALLEN|    30|1600|               2|
|TURNER|    30|1500|               3|
|  WARD|    30|1250|               4|
|MARTIN|    30|1250|               4|
| JAMES|    30| 950|               6|
+------+------+----+----------------+



In [174]:
from pyspark.sql.functions import row_number
new_df = df.withColumn( 'dept_salary_rank', row_number().over(dept_window_spec))
new_df.select(
    'ename', 'deptno', 'sal', 'dept_salary_rank'
).show()

+------+------+----+----------------+
| ename|deptno| sal|dept_salary_rank|
+------+------+----+----------------+
| SCOTT|    20|3000|               1|
|  FORD|    20|3000|               2|
| JONES|    20|2975|               3|
| ADAMS|    20|1100|               4|
| SMITH|    20| 800|               5|
|  KING|    10|5000|               1|
| CLARK|    10|2450|               2|
|MILLER|    10|1300|               3|
|  JACK|    70|3200|               1|
| BLAKE|    30|2850|               1|
| ALLEN|    30|1600|               2|
|TURNER|    30|1500|               3|
|  WARD|    30|1250|               4|
|MARTIN|    30|1250|               5|
| JAMES|    30| 950|               6|
+------+------+----+----------------+



In [175]:
from pyspark.sql.functions import dense_rank
new_df = df.withColumn( 'dept_salary_rank', dense_rank().over(dept_window_spec))
new_df.select(
    'ename', 'deptno', 'sal', 'dept_salary_rank'
).show()

+------+------+----+----------------+
| ename|deptno| sal|dept_salary_rank|
+------+------+----+----------------+
| SCOTT|    20|3000|               1|
|  FORD|    20|3000|               1|
| JONES|    20|2975|               2|
| ADAMS|    20|1100|               3|
| SMITH|    20| 800|               4|
|  KING|    10|5000|               1|
| CLARK|    10|2450|               2|
|MILLER|    10|1300|               3|
|  JACK|    70|3200|               1|
| BLAKE|    30|2850|               1|
| ALLEN|    30|1600|               2|
|TURNER|    30|1500|               3|
|  WARD|    30|1250|               4|
|MARTIN|    30|1250|               4|
| JAMES|    30| 950|               5|
+------+------+----+----------------+



In [176]:
#누적 급여 계산#누적 급여 계산
from pyspark.sql.functions import dense_rank
sum_window_spec = Window.partitionBy('deptno').orderBy('empno')

In [177]:
new_df = df.withColumn( 'cum_salary', sum('sal').over(sum_window_spec))
new_df.select(
    'ename', 'deptno', 'sal', 'cum_salary'
).show()

+------+------+----+----------+
| ename|deptno| sal|cum_salary|
+------+------+----+----------+
| SMITH|    20| 800|       800|
| JONES|    20|2975|      3775|
| SCOTT|    20|3000|      6775|
| ADAMS|    20|1100|      7875|
|  FORD|    20|3000|     10875|
| CLARK|    10|2450|      2450|
|  KING|    10|5000|      7450|
|MILLER|    10|1300|      8750|
|  JACK|    70|3200|      3200|
| ALLEN|    30|1600|      1600|
|  WARD|    30|1250|      2850|
|MARTIN|    30|1250|      4100|
| BLAKE|    30|2850|      6950|
|TURNER|    30|1500|      8450|
| JAMES|    30| 950|      9400|
+------+------+----+----------+



In [178]:
from pyspark.sql.functions import avg

# 부서별 평균 급여 계산
df.groupBy("deptno").agg(avg("sal").alias("avg_salary")).show()


+------+------------------+
|deptno|        avg_salary|
+------+------------------+
|    20|            2175.0|
|    10|2916.6666666666665|
|    70|            3200.0|
|    30|1566.6666666666667|
+------+------------------+



In [179]:
from pyspark.sql.functions import dense_rank
avg_window_spec = Window.partitionBy('deptno')
new_avg_df = df.withColumn('dept_avg_salary', avg('sal').over(avg_window_spec))
new_avg_df.select('empno', 'deptno', 'dept_avg_salary',).show()

+-----+------+------------------+
|empno|deptno|   dept_avg_salary|
+-----+------+------------------+
| 7369|    20|            2175.0|
| 7566|    20|            2175.0|
| 7788|    20|            2175.0|
| 7876|    20|            2175.0|
| 7902|    20|            2175.0|
| 7782|    10|2916.6666666666665|
| 7839|    10|2916.6666666666665|
| 7934|    10|2916.6666666666665|
| 9292|    70|            3200.0|
| 7499|    30|1566.6666666666667|
| 7521|    30|1566.6666666666667|
| 7654|    30|1566.6666666666667|
| 7698|    30|1566.6666666666667|
| 7844|    30|1566.6666666666667|
| 7900|    30|1566.6666666666667|
+-----+------+------------------+



In [180]:
#lead, lag 이전 급여 , 이후 급여
from pyspark.sql.functions import lag, lead

row_window_spec = Window.partitionBy('deptno').orderBy('empno')

#이전급여 컬럼, 이후급여 컬럼 2개 추가
lead_lagg_sal_df = df.withColumn('prev_salary', lag('sal').over(row_window_spec))\
            .withColumn('next_salary', lead('sal').over(row_window_spec))

lead_lagg_sal_df.select('ename', 'deptno', 'sal',  'prev_salary', 'next_salary').show()

+------+------+----+-----------+-----------+
| ename|deptno| sal|prev_salary|next_salary|
+------+------+----+-----------+-----------+
| SMITH|    20| 800|       null|       2975|
| JONES|    20|2975|        800|       3000|
| SCOTT|    20|3000|       2975|       1100|
| ADAMS|    20|1100|       3000|       3000|
|  FORD|    20|3000|       1100|       null|
| CLARK|    10|2450|       null|       5000|
|  KING|    10|5000|       2450|       1300|
|MILLER|    10|1300|       5000|       null|
|  JACK|    70|3200|       null|       null|
| ALLEN|    30|1600|       null|       1250|
|  WARD|    30|1250|       1600|       1250|
|MARTIN|    30|1250|       1250|       2850|
| BLAKE|    30|2850|       1250|       1500|
|TURNER|    30|1500|       2850|        950|
| JAMES|    30| 950|       1500|       null|
+------+------+----+-----------+-----------+



In [182]:
df.groupBy('deptno', 'job').agg(count('*'),sum('sal')).show()

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|    20|  ANALYST|       2|    6000|
|    20|  MANAGER|       1|    2975|
|    30|  MANAGER|       1|    2850|
|    70|    CLERK|       1|    3200|
|    30| SALESMAN|       4|    5600|
|    30|    CLERK|       1|     950|
|    20|    CLERK|       2|    1900|
|    10|PRESIDENT|       1|    5000|
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
+------+---------+--------+--------+



In [183]:
df.rollup('deptno', 'job').agg(count('*'), sum('sal')).orderBy('deptno', 'job').show()

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|  null|     null|      15|   32225|
|    10|     null|       3|    8750|
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|     null|       5|   10875|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
|    20|  MANAGER|       1|    2975|
|    30|     null|       6|    9400|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    5600|
|    70|     null|       1|    3200|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [184]:
df.cube('deptno', 'job').agg(count('*'), sum('sal')).orderBy('deptno', 'job').show()

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|  null|     null|      15|   32225|
|  null|  ANALYST|       2|    6000|
|  null|    CLERK|       5|    7350|
|  null|  MANAGER|       3|    8275|
|  null|PRESIDENT|       1|    5000|
|  null| SALESMAN|       4|    5600|
|    10|     null|       3|    8750|
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|     null|       5|   10875|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
|    20|  MANAGER|       1|    2975|
|    30|     null|       6|    9400|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    5600|
|    70|     null|       1|    3200|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



select
    deptno, job,
    max(sal)as max_sal,
    min(sal)as min_sal,
from emp
group by rollup(deptno,  job)
order by deptno, job;

In [188]:
df.rollup('deptno', 'job').agg(max('sal'),min('sal').alias('min_sal')).orderBy('deptno','job').show()

+------+---------+--------+-------+
|deptno|      job|max(sal)|min_sal|
+------+---------+--------+-------+
|  null|     null|    5000|    800|
|    10|     null|    5000|   1300|
|    10|    CLERK|    1300|   1300|
|    10|  MANAGER|    2450|   2450|
|    10|PRESIDENT|    5000|   5000|
|    20|     null|    3000|    800|
|    20|  ANALYST|    3000|   3000|
|    20|    CLERK|    1100|    800|
|    20|  MANAGER|    2975|   2975|
|    30|     null|    2850|    950|
|    30|    CLERK|     950|    950|
|    30|  MANAGER|    2850|   2850|
|    30| SALESMAN|    1600|   1250|
|    70|     null|    3200|   3200|
|    70|    CLERK|    3200|   3200|
+------+---------+--------+-------+



In [ ]:
# job별 평균급여 job, avg_salary, total_salary

In [ ]:
#dpetno, job 평균급여, 최대급여를모두 소계를 냅니다.

In [189]:
result = df.rollup("deptno", "job").agg(
    avg("sal").alias("avg_salary"),  # 평균 급여
    max("sal").alias("max_salary")  # 최대 급여
)

# 결과 출력
result.show()

+------+---------+------------------+----------+
|deptno|      job|        avg_salary|max_salary|
+------+---------+------------------+----------+
|    10|    CLERK|            1300.0|      1300|
|    20|  ANALYST|            3000.0|      3000|
|    70|     null|            3200.0|      3200|
|    30| SALESMAN|            1400.0|      1600|
|    10|     null|2916.6666666666665|      5000|
|    70|    CLERK|            3200.0|      3200|
|    30|  MANAGER|            2850.0|      2850|
|  null|     null|2148.3333333333335|      5000|
|    10|PRESIDENT|            5000.0|      5000|
|    30|     null|1566.6666666666667|      2850|
|    10|  MANAGER|            2450.0|      2450|
|    20|     null|            2175.0|      3000|
|    20|  MANAGER|            2975.0|      2975|
|    20|    CLERK|             950.0|      1100|
|    30|    CLERK|             950.0|       950|
+------+---------+------------------+----------+



In [190]:
spark.stop()